# 빌딩 샘플링하기

1) 전체 4542개의 초고층빌딩(20층 이상) 중 400개의 샘플 데이터 무작위 선정 
- random.choice 매서드 사용
- 샘플로 뽑힌 빌딩 400개는 이하 spl로 칭함

2) spl 각각 가장 가까운 2개의 빌딩 구하는 함수 생성
- Key 정보가 담긴 데이터 프레임 구하는 함수

3) spl의 가장 가까운 두 쌍 구하고 중복값 제거 후 저장

4) 각각의 빌딩쌍 정보가 담긴 데이터프레임 만들어 저장

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
from haversine import *

# 1) 빌딩 샘플 선정

In [2]:
# 데이터 업로드 후 확인
data = gpd.read_file("../data/buildings/high_bld_over20_include_latlon.shp",encoding="euc-kr")
data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 4542 entries, 0 to 4541
Data columns (total 43 columns):
BDTYP_CD      4542 non-null object
BD_MGT_SN     4542 non-null object
BSI_INT_SN    4542 non-null float64
BSI_ZON_NO    4542 non-null object
BULD_MNNM     4542 non-null int64
BULD_NM       4097 non-null object
BULD_NM_DC    3756 non-null object
BULD_SE_CD    4542 non-null object
BULD_SLNO     4542 non-null int64
BUL_DPN_SE    4542 non-null object
BUL_MAN_NO    4542 non-null int64
EMD_CD        4542 non-null object
GRO_FLO_CO    4542 non-null int64
LI_CD         4542 non-null object
LNBR_MNNM     4542 non-null int64
LNBR_SLNO     4542 non-null int64
MNTN_YN       4542 non-null object
RN_CD         4542 non-null object
SIG_CD        4542 non-null object
UND_FLO_CO    4542 non-null int64
ZIP           0 non-null object
X_AXIS        4542 non-null int64
Y_AXIS        4542 non-null int64
HOUS_ID       4542 non-null object
TOTAL_AREA    4542 non-null float64
ADMI_CD       4542 n

In [5]:
# BD_MGT_SN을 kEY로 사용할 수 있는지 확인
print("전체 데이터 수 :",len(data))
print("BD_MGT_SN 중복값 제거 수 :",len(data.BD_MGT_SN.unique()))

전체 데이터 수 : 4542
BD_MGT_SN 중복값 제거 수 : 4542


In [6]:
# 랜덤 빌딩 400개 선정
np.random.seed(0)
random_bld_list = numpy.random.choice(data.BD_MGT_SN, size=400, replace=False, p=None)
random_bld_list

array(['1168010700106320000000002', '1159010100103250000033151',
       '1171010200100070017000139', '1156013300106070001037549',
       '1153010600103340000014364', '1144011400104370000008970',
       '1165010200100110017000001', '1156010800100460001033488',
       '1150010300110030019019280', '1120010500108180000022228',
       '1156013200149340000037525', '1150010300110250033029436',
       '1126010300104500000014365', '1130510200104100000040059',
       '1171010700101650000000007', '1114011400100230000022809',
       '1165010800116780001023258', '1156012900100760000020141',
       '1123010500101110000022149', '1168010600106330000014415',
       '1156011200101640000029599', '1171010100100190000000050',
       '1135010500112760000006283', '1171010700101400000005030',
       '1174010900104470017000001', '1168011800105270000000027',
       '1150010300110250033029465', '1171010400101620000012081',
       '1165010700100200004000002', '1162010200117300000000023',
       '11710101001003500

In [7]:
# 샘플 데이터 지정
bld_spl = data.set_index("BD_MGT_SN").loc[random_bld_list].reset_index()
bld_spl

,BD_MGT_SN,BDTYP_CD,BSI_INT_SN,BSI_ZON_NO,BULD_MNNM,BULD_NM,BULD_NM_DC,BULD_SE_CD,BULD_SLNO,BUL_DPN_SE,...,XMAX,YMAX,SHAPE_AREA,lat_center,lon_center,lat_max,lon_max,lat_min,lon_min,geometry
0,1168010700106320000000002,02001,11751.0,06020,78,압구정 하이츠 파크,에이동,0,0,M,...,202988,447138,623.54,37.523519,127.033666,37.523727,127.033802,37.523312,127.033530,"POLYGON ((202980.006 447125.272, 202980.272 44..."
1,1159010100103250000033151,02001,19498.0,06920,19,신동아리버파크,702동,0,0,S,...,195036,445316,538.48,37.507076,126.943631,37.507302,126.943857,37.506851,126.943405,"POLYGON ((195035.244 445307.768, 195026.632 44..."
2,1171010200100070017000139,02001,3371.0,05510,562,"한빛프라쟈,삼성웰리스아파트",None,0,0,M,...,208836,446233,1411.65,37.515319,127.099505,37.515535,127.099947,37.515130,127.099007,"POLYGON ((208833.497 446217.358, 208776.918 44..."
3,1156013300106070001037549,02001,25647.0,07376,2,대림동코오롱아파트,2동,0,5,M,...,190727,445347,507.37,37.507358,126.894907,37.507548,126.895121,37.507169,126.894704,"POLYGON ((190708.019 445335.041, 190717.498 44..."
4,1153010600103340000014364,02001,1426.0,08275,390,벽산블루밍,204동,0,0,M,...,188093,443931,567.62,37.494633,126.865060,37.494759,126.865354,37.494506,126.864766,"POLYGON ((188088.651 443929.803, 188092.116 44..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,1126010600108170000000015,02001,35403.0,02055,165,신내우디안2단지,215동,0,0,M,...,209793,457307,878.08,37.615122,127.110545,37.615301,127.110919,37.615005,127.110262,"POLYGON ((209736.451 457306.024, 209740.647 45..."
396,1121510300106310001010199,02001,2637.0,05119,29,현대프라임아파트,13동,0,0,S,...,208654,448584,397.78,37.536575,127.097734,37.536719,127.097916,37.536431,127.097565,"POLYGON ((208653.575 448575.363, 208649.815 44..."
397,1154510300110130000017492,02001,3068.0,08646,816,벽산아파트,506동,0,0,M,...,192957,438798,498.41,37.448389,126.920180,37.448560,126.920406,37.448218,126.919943,"POLYGON ((192916.926 438788.648, 192924.474 43..."
398,1156011400100960000000007,02001,19291.0,07221,597,당산동 반도유보라팰리스,106동,0,0,M,...,191379,447501,549.67,37.526799,126.902290,37.526962,126.902470,37.526655,126.902109,"POLYGON ((191372.875 447497.483, 191371.979 44..."


In [ ]:
# 샘플로 지정한 데이터 1차 저장
bld_spl.to_file("../data/buildings/building_sample.shp",encoding="euc-kr")

# 2) 가장 가까운 빌딩 구하는 함수 생성

In [8]:
spl = bld_spl[["BD_MGT_SN","BULD_NM","lat_center","lon_center"]]
spl.head()

,BD_MGT_SN,BULD_NM,lat_center,lon_center
0,1168010700106320000000002,압구정 하이츠 파크,37.523519,127.033666
1,1159010100103250000033151,신동아리버파크,37.507076,126.943631
2,1171010200100070017000139,"한빛프라쟈,삼성웰리스아파트",37.515319,127.099505
3,1156013300106070001037549,대림동코오롱아파트,37.507358,126.894907
4,1153010600103340000014364,벽산블루밍,37.494633,126.865060


In [9]:
all_b = data[["BD_MGT_SN","BULD_NM","lat_center","lon_center"]]
all_b.head()

,BD_MGT_SN,BULD_NM,lat_center,lon_center
0,1111012000101160000028381,에스타워,37.569797,126.973931
1,1111012000101630000031927,광화문오피시아빌딩,37.569698,126.974905
2,1111011700100290000000001,포시즌스호텔,37.570698,126.975402
3,1111012000102260000000001,흥국생명빌딩,37.569643,126.972143
4,1111012000101150000000001,금호아시아나본관(메인타워),37.569796,126.973173


In [10]:
# 데이터 프레임 생성 함수
def find_closest(lat,lon) :
    dis = []
    name = []
    code = []
    for i in range(0,len(all_b)): 
        lyon = (lat, lon) 
        pairs = (all_b.loc[i,"lat_center"], all_b.loc[i,"lon_center"])
        a = haversine(lyon, pairs)
        b = all_b.loc[i,"BD_MGT_SN"]
        c = all_b.loc[i,"BULD_NM"]
        dis.append(a)
        code.append(b)
        name.append(c)
    result = pd.DataFrame({"빌딩코드":code, "빌딩명":name, "거리(km)":dis}).sort_values("거리(km)").iloc[1:3].reset_index()
    return result

In [11]:
# 확인해보기
lat1 = spl.loc[0,"lat_center"]
lon1 = spl.loc[0,"lon_center"]
find_closest(lat1,lon1)

,index,빌딩코드,빌딩명,거리(km)
0,4513,1168010700106320000000001,압구정 하이츠 파크,0.059186
1,3063,1168010800101050007005368,두산빌딩,0.479457


In [12]:
# 전체 구해서 돌리기
for n in range(0,400) :
    lat = spl.loc[n,"lat_center"]
    lon = spl.loc[n,"lon_center"]
    print("%d. %s(빌딩코드:%s) 의 가장 가까운 두 빌딩" %(n,spl.loc[n,"BULD_NM"],spl.loc[n,"BD_MGT_SN"]))
    print(find_closest(lat,lon),"\n\n")

0. 압구정 하이츠 파크(빌딩코드:1168010700106320000000002) 의 가장 가까운 두 빌딩
   index                       빌딩코드         빌딩명    거리(km)
0   4513  1168010700106320000000001  압구정 하이츠 파크  0.059186
1   3063  1168010800101050007005368        두산빌딩  0.479457 


1. 신동아리버파크(빌딩코드:1159010100103250000033151) 의 가장 가까운 두 빌딩
   index                       빌딩코드      빌딩명    거리(km)
0   2554  1159010100103250000033149  신동아리버파크  0.044263
1   2555  1159010100103250000033150  신동아리버파크  0.048099 


2. 한빛프라쟈,삼성웰리스아파트(빌딩코드:1171010200100070017000139) 의 가장 가까운 두 빌딩
   index                       빌딩코드       빌딩명    거리(km)
0   3837  1171010200100070025000226    월드타워빌딩  0.030500
1   3499  1171010200100070011000267  한국광고문화회관  0.071579 


3. 대림동코오롱아파트(빌딩코드:1156013300106070001037549) 의 가장 가까운 두 빌딩
   index                       빌딩코드        빌딩명    거리(km)
0   2406  1156013300106070001037548  대림동코오롱아파트  0.041359
1   2408  1156013300106070001037550  대림동코오롱아파트  0.046259 


4. 벽산블루밍(빌딩코드:1153010600103340000014364) 의 가장 가까운 두 빌딩
   index       

# 3) 가장 가까운 두 쌍 모음의 중복값 제거 후 저장

In [14]:
# 전체 짝 구하기
total = []
for n in range(0,400) :
    lat = spl.loc[n,"lat_center"]
    lon = spl.loc[n,"lon_center"]
    df = find_closest(lat,lon)
    a = df.loc[0,"index"]
    b = df.loc[1,"index"]
    code = spl.loc[n,"BD_MGT_SN"]
    c = all_b[all_b.BD_MGT_SN == code].reset_index().iloc[0,0]
    aa = [c,a]
    bb = [c,b]
    total.append(aa)
    total.append(bb)

In [16]:
total

[[2962, 4513],
 [2962, 3063],
 [2556, 2554],
 [2556, 2555],
 [3386, 3837],
 [3386, 3499],
 [2407, 2406],
 [2407, 2408],
 [2110, 2112],
 [2110, 2121],
 [1553, 1556],
 [1553, 3724],
 [2835, 3829],
 [2835, 2742],
 [2423, 2422],
 [2423, 2311],
 [1869, 1898],
 [1869, 1868],
 [308, 311],
 [308, 314],
 [2343, 4085],
 [2343, 2346],
 [1829, 1817],
 [1829, 4522],
 [849, 4101],
 [849, 4280],
 [1172, 3889],
 [1172, 1173],
 [3380, 3373],
 [3380, 3381],
 [72, 43],
 [72, 61],
 [2837, 4154],
 [2837, 2705],
 [2475, 3743],
 [2475, 2469],
 [716, 4326],
 [716, 4133],
 [2906, 2905],
 [2906, 2907],
 [2385, 2383],
 [2385, 4532],
 [3214, 3215],
 [3214, 3216],
 [1362, 1364],
 [1362, 1363],
 [3367, 3859],
 [3367, 4441],
 [3471, 3486],
 [3471, 3695],
 [3086, 3099],
 [3086, 3084],
 [1815, 1831],
 [1815, 1804],
 [3555, 3561],
 [3555, 3559],
 [2873, 2874],
 [2873, 2762],
 [4151, 2605],
 [4151, 3606],
 [3149, 3152],
 [3149, 3166],
 [1834, 1814],
 [1834, 1811],
 [1493, 1494],
 [1493, 1492],
 [3363, 3740],
 [3363, 336

In [17]:
# str로 변환해서 중복값 제거
for i in total :
    i.sort()
total.sort()
total_str = []
for i in total :
    total_str.append(str(i))
print("전체 빌딩 쌍 갯수 :",len(total_str))
print("중복 값 제거한 빌딩 쌍 갯수 :",len(set(total_str)))

전체 빌딩 쌍 갯수 : 800
중복 값 제거한 빌딩 쌍 갯수 : 775


In [26]:
# 다시 list화 / 빌딩 하나하나 저장해보기
bld_smpls = []
bld_smpls_sep = []
for i in set(total_str) :
    nums = i.replace("[","").replace("]","").split(", ")
    bld_smpls.append([int(nums[0]),int(nums[1])])
    bld_smpls_sep.append(int(nums[0]))
    bld_smpls_sep.append(int(nums[1]))

In [27]:
# 최종 확인
print("빌딩 쌍 갯수 :",len(bld_smpls))
print("빌딩 갯수 :",len(set(bld_smpls_sep)))

빌딩 쌍 갯수 : 775
빌딩 갯수 : 1072


In [30]:
bld_smpls.sort()
print("순서대로 정렬한 빌딩쌍 :",bld_smpls[0:8]," ... ")

순서대로 정렬한 빌딩쌍 : [[40, 42], [42, 4519], [43, 72], [45, 54], [45, 55], [56, 57], [57, 58], [61, 72]]  ... 


In [31]:
# 샘플 데이터 전체 컬럼 뽑아내기
data_2 = data.reset_index()
data_3 = data_2.loc[bld_smpls_sep].drop_duplicates()
data_3.sort_values("index").head(3)

,index,BDTYP_CD,BD_MGT_SN,BSI_INT_SN,BSI_ZON_NO,BULD_MNNM,BULD_NM,BULD_NM_DC,BULD_SE_CD,BULD_SLNO,...,XMAX,YMAX,SHAPE_AREA,lat_center,lon_center,lat_max,lon_max,lat_min,lon_min,geometry
40,40,04402,1114011400101500000022693,13730.0,04513,55,삼성생명빌딩,본사(삼성프라자),0,0,...,197824,451375,2673.05,37.561616,126.974975,37.561904,126.975371,37.561363,126.974545,"POLYGON ((197823.631 451329.587, 197823.741 45..."
42,42,04402,1114016600101110000000001,13739.0,04514,73,태평로빌딩,None,0,0,...,197896,451514,1084.26,37.562967,126.975960,37.563157,126.976186,37.562769,126.975733,"POLYGON ((197866.622 451513.504, 197884.438 45..."
43,43,04402,1114011400100430000023455,13748.0,04525,92,한화금융센터_태평로,None,0,0,...,198043,451665,1917.79,37.564229,126.977566,37.564518,126.977849,37.563941,126.977317,"POLYGON ((198042.112 451656.554, 198028.468 45..."


In [ ]:
# 저장하기
data_3.to_file("../data/2014_building/building_samples_sep.shp",encoding="euc-kr")

# 4) 각각의 빌딩쌍 정보가 담긴 데이터프레임 만들어 저장

In [32]:
bld_1 = []
bld_2 = []
code_1 = []
code_2 = []
distance = []

for i in bld_smpls :
    index_1 = i[0]
    index_2 = i[1]
    BULD_NM_1 = all_b.loc[index_1,"BULD_NM"]
    BULD_NM_2 = all_b.loc[index_2,"BULD_NM"]
    BD_MGT_SN_1 = all_b.loc[index_1,"BD_MGT_SN"]
    BD_MGT_SN_2 = all_b.loc[index_2,"BD_MGT_SN"]
    a = (all_b.loc[index_1,"lat_center"],all_b.loc[index_1,"lon_center"])
    b = (all_b.loc[index_2,"lat_center"],all_b.loc[index_2,"lon_center"])
    dis = haversine(a,b)
    bld_1.append(BULD_NM_1)
    bld_2.append(BULD_NM_2)
    code_1.append(BD_MGT_SN_1)
    code_2.append(BD_MGT_SN_2)
    distance.append(dis)

match_df = pd.DataFrame({"빌딩1":bld_1,"빌딩2":bld_2,"빌딩코드1":code_1,"빌딩코드2":code_2,
                        "중심간거리(km)":distance})
match_df

,빌딩1,빌딩2,빌딩코드1,빌딩코드2,중심간거리(km)
0,삼성생명빌딩,태평로빌딩,1114011400101500000022693,1114016600101110000000001,0.173543
1,태평로빌딩,삼성본관,1114016600101110000000001,1114011400102500000022814,0.073892
2,한화금융센터_태평로,서울프라자호텔,1114011400100430000023455,1114011400100230000022809,0.066473
3,서울스퀘어,서울시티타워,1114011800105410000021354,1114011800105810000021764,0.136733
4,서울스퀘어,힐튼호텔,1114011800105410000021354,1114011800103950000021505,0.153754
...,...,...,...,...,...
770,당산동 반도유보라팰리스,당산동 반도유보라팰리스,1156011400100960000000006,1156011400100960000000005,0.039143
771,두산위브아파트,두산위브아파트,1123010500109840003041116,1123010500109840003041121,0.009220
772,홍은 센트레빌,홍은동풍림아이원아파트,1141011800104610001000001,1141011800104600000012964,1.153017
773,삼각산아이원아파트,삼각산아이원아파트,1130510100113570000035086,1130510100113570000035082,0.050228


In [33]:
# 저장하기
match_df.to_csv("../data/buildings/빌딩쌍데이터프레임.csv",encoding="euc-kr")